# Benin Solar EDA - Exploratory Data Analysis
## Malanville Solar Site Analysis

This notebook performs comprehensive exploratory data analysis on solar irradiance and meteorological data from the Malanville site in Benin.

## 1. Imports and Configuration

In [1]:
print(1)

1


## 2. Configuration and File Paths

In [ ]:
# Configuration
RAW_FILE_PATH = '../data/benin-malanville.csv'
DAYTIME_GHI_THRESHOLD = 5  # W/m^2

# Verify file exists
print(f"Current working directory: {os.getcwd()}")
print(f"Looking for data file at: {RAW_FILE_PATH}")

if os.path.exists(RAW_FILE_PATH):
    file_size_mb = os.path.getsize(RAW_FILE_PATH) / (1024 * 1024)
    print(f"✓ Data file found ({file_size_mb:.2f} MB)")
else:
    print(f"✗ ERROR: Data file not found at {RAW_FILE_PATH}")
    print(f"Available files in ../data/:")
    if os.path.exists('../data'):
        print(os.listdir('../data'))
    else:
        print("../data directory does not exist")

## 3. Data Loading and Cleaning Functions

In [ ]:
def load_and_clean_data(filepath):
    """
    Load and clean solar data from CSV file.
    
    Parameters:
    -----------
    filepath : str
        Path to the CSV file
    
    Returns:
    --------
    pd.DataFrame or None
        Cleaned dataframe with timestamp index, or None if error occurs
    """
    try:
        # Load CSV with optimized dtypes
        print(f"Loading data from {filepath}...")
        df = pd.read_csv(filepath)
        print(f"✓ Loaded {len(df):,} rows and {len(df.columns)} columns")
        
        # Convert timestamp and set as index
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df = df.set_index('Timestamp')
        print(f"✓ Timestamp set as index")
        print(f"  Date range: {df.index.min()} to {df.index.max()}")
        
        # Impute weather-related columns with their median
        impute_cols = ['Tamb', 'RH', 'WS', 'WSgust', 'WSstdev', 'WD', 'WDstdev', 'BP', 'TModA', 'TModB']
        print(f"\nImputing missing values...")
        for col in impute_cols:
            if col in df.columns:
                missing_count = df[col].isna().sum()
                if missing_count > 0:
                    median_val = df[col].median()
                    df[col] = df[col].fillna(median_val)
                    pct = (missing_count / len(df)) * 100
                    print(f"  - {col}: {missing_count:,} missing values ({pct:.2f}%) → imputed with median {median_val:.2f}")
        
        print(f"\n✓ Data cleaning completed")
        print(f"  Final shape: {df.shape}")
        print(f"  Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        
        return df
        
    except FileNotFoundError:
        print(f"✗ ERROR: File not found at {filepath}")
        return None
    except Exception as e:
        print(f"✗ ERROR: Could not load or parse CSV: {e}")
        return None

print("✓ Functions defined successfully")

## 4. Load and Process Data

In [ ]:
# Load and clean the data
df = load_and_clean_data(RAW_FILE_PATH)

if df is None:
    print("\n✗ Failed to load data. Stopping execution.")
    sys.exit(1)
else:
    print("\n" + "="*70)
    print("DATA LOADED SUCCESSFULLY")
    print("="*70)

## 5. Data Overview

In [ ]:
print("\n" + "="*70)
print("DATASET OVERVIEW")
print("="*70)

print(f"\nShape: {df.shape}")
print(f"\nDate Range: {df.index.min()} to {df.index.max()}")
print(f"Duration: {(df.index.max() - df.index.min()).days} days")

print(f"\nColumn Names and Types:")
print(df.dtypes)

print(f"\nFirst 5 rows:")
print(df.head())

print(f"\nLast 5 rows:")
print(df.tail())

## 6. Missing Values Analysis

In [ ]:
print("\n" + "="*70)
print("MISSING VALUES ANALYSIS")
print("="*70)

missing_values = df.isna().sum()
missing_percent = (missing_values / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Count': missing_values.values,
    'Missing %': missing_percent.values
}).sort_values('Missing %', ascending=False)

print("\n" + missing_df.to_string(index=False))

if missing_df['Missing %'].max() == 0:
    print("\n✓ No missing values remaining after imputation!")
else:
    cols_with_missing = missing_df[missing_df['Missing %'] > 0]
    print(f"\n⚠ {len(cols_with_missing)} columns still have missing values")

## 7. Summary Statistics

In [ ]:
print("\n" + "="*70)
print("SUMMARY STATISTICS")
print("="*70)

print("\n" + df.describe().to_string())

## 8. Daytime Data Filter

In [ ]:
# Filter for daytime data (GHI > threshold)
df_daytime = df[df['GHI'] > DAYTIME_GHI_THRESHOLD].copy()

print("\n" + "="*70)
print("DAYTIME DATA ANALYSIS")
print("="*70)

print(f"\nThreshold: GHI > {DAYTIME_GHI_THRESHOLD} W/m²")
print(f"Total records: {len(df):,}")
print(f"Daytime records: {len(df_daytime):,}")
print(f"Percentage: {len(df_daytime)/len(df)*100:.2f}%")
print(f"Nighttime records: {len(df) - len(df_daytime):,}")

## 9. Data Visualizations

In [ ]:
# Create comprehensive visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Solar Irradiance and Environmental Conditions - Distribution Analysis', fontsize=14, fontweight='bold')

# Plot 1: GHI Distribution (Daytime)
axes[0, 0].hist(df_daytime['GHI'], bins=50, color='orange', edgecolor='black', alpha=0.7)
axes[0, 0].set_title('GHI Distribution (Daytime)', fontweight='bold')
axes[0, 0].set_xlabel('GHI (W/m²)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Temperature Distribution
axes[0, 1].hist(df['Tamb'], bins=50, color='red', edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Ambient Temperature Distribution', fontweight='bold')
axes[0, 1].set_xlabel('Temperature (°C)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Wind Speed Distribution
axes[1, 0].hist(df['WS'], bins=50, color='blue', edgecolor='black', alpha=0.7)
axes[1, 0].set_title('Wind Speed Distribution', fontweight='bold')
axes[1, 0].set_xlabel('Wind Speed (m/s)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Relative Humidity Distribution
axes[1, 1].hist(df['RH'], bins=50, color='green', edgecolor='black', alpha=0.7)
axes[1, 1].set_title('Relative Humidity Distribution', fontweight='bold')
axes[1, 1].set_xlabel('Relative Humidity (%)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Distribution plots created")

## 10. Relationship Analysis - Scatter Plots

In [ ]:
# Scatter plots for relationship analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Relationship Analysis - Daytime Data', fontsize=14, fontweight='bold')

# Sample data for better visualization (if dataset is large)
sample_size = min(5000, len(df_daytime))
df_sample = df_daytime.sample(n=sample_size, random_state=42)

# Plot 1: GHI vs Temperature
axes[0, 0].scatter(df_sample['Tamb'], df_sample['GHI'], alpha=0.5, s=10, color='orange')
axes[0, 0].set_title('GHI vs Ambient Temperature', fontweight='bold')
axes[0, 0].set_xlabel('Temperature (°C)')
axes[0, 0].set_ylabel('GHI (W/m²)')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: GHI vs Wind Speed
axes[0, 1].scatter(df_sample['WS'], df_sample['GHI'], alpha=0.5, s=10, color='blue')
axes[0, 1].set_title('GHI vs Wind Speed', fontweight='bold')
axes[0, 1].set_xlabel('Wind Speed (m/s)')
axes[0, 1].set_ylabel('GHI (W/m²)')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: GHI vs Relative Humidity
axes[1, 0].scatter(df_sample['RH'], df_sample['GHI'], alpha=0.5, s=10, color='green')
axes[1, 0].set_title('GHI vs Relative Humidity', fontweight='bold')
axes[1, 0].set_xlabel('Relative Humidity (%)')
axes[1, 0].set_ylabel('GHI (W/m²)')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Module Temperature vs Ambient Temperature
axes[1, 1].scatter(df_sample['Tamb'], df_sample['TModA'], alpha=0.5, s=10, color='red')
axes[1, 1].set_title('Module Temperature vs Ambient Temperature', fontweight='bold')
axes[1, 1].set_xlabel('Ambient Temperature (°C)')
axes[1, 1].set_ylabel('Module Temperature (°C)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✓ Scatter plots created (using {sample_size:,} samples)")

## 11. Correlation Analysis

In [ ]:
# Select numeric columns for correlation
numeric_cols = df_daytime.select_dtypes(include=[np.number]).columns.tolist()

# Calculate correlation matrix
corr_matrix = df_daytime[numeric_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix - Daytime Data', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("✓ Correlation heatmap created")

# Print top correlations with GHI
print("\n" + "="*70)
print("TOP CORRELATIONS WITH GHI (Solar Irradiance)")
print("="*70)

ghi_corr = corr_matrix['GHI'].sort_values(ascending=False)
print("\n" + ghi_corr.to_string())

## 12. Time Series Analysis

In [ ]:
# Resample to daily mean for better visualization
df_daily = df.resample('D').mean(numeric_only=True)

fig, axes = plt.subplots(3, 1, figsize=(16, 12))
fig.suptitle('Time Series Analysis - Daily Averages', fontsize=14, fontweight='bold')

# Plot 1: Daily Mean GHI
axes[0].plot(df_daily.index, df_daily['GHI'], color='orange', linewidth=1.5, label='GHI')
axes[0].fill_between(df_daily.index, df_daily['GHI'], alpha=0.3, color='orange')
axes[0].set_title('Daily Mean Global Horizontal Irradiance (GHI)', fontweight='bold')
axes[0].set_ylabel('GHI (W/m²)')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Plot 2: Daily Mean Temperature
axes[1].plot(df_daily.index, df_daily['Tamb'], color='red', linewidth=1.5, label='Ambient Temperature')
axes[1].fill_between(df_daily.index, df_daily['Tamb'], alpha=0.3, color='red')
axes[1].set_title('Daily Mean Ambient Temperature', fontweight='bold')
axes[1].set_ylabel('Temperature (°C)')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

# Plot 3: Daily Mean Wind Speed
axes[2].plot(df_daily.index, df_daily['WS'], color='blue', linewidth=1.5, label='Wind Speed')
axes[2].fill_between(df_daily.index, df_daily['WS'], alpha=0.3, color='blue')
axes[2].set_title('Daily Mean Wind Speed', fontweight='bold')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Wind Speed (m/s)')
axes[2].grid(True, alpha=0.3)
axes[2].legend()

plt.tight_layout()
plt.show()

print("✓ Time series plots created")

## 13. Module Output Analysis

In [ ]:
if 'ModA' in df.columns and 'ModB' in df.columns:
    print("\n" + "="*70)
    print("MODULE OUTPUT ANALYSIS")
    print("="*70)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    fig.suptitle('Module Power Output Analysis', fontsize=14, fontweight='bold')
    
    # Plot 1: Module A vs GHI
    sample_size = min(5000, len(df_daytime))
    df_sample = df_daytime.sample(n=sample_size, random_state=42)
    
    axes[0].scatter(df_sample['GHI'], df_sample['ModA'], alpha=0.5, s=10, color='purple')
    axes[0].set_title('Module A Output vs GHI', fontweight='bold')
    axes[0].set_xlabel('GHI (W/m²)')
    axes[0].set_ylabel('Module A Output (W)')
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Module B vs GHI
    axes[1].scatter(df_sample['GHI'], df_sample['ModB'], alpha=0.5, s=10, color='brown')
    axes[1].set_title('Module B Output vs GHI', fontweight='bold')
    axes[1].set_xlabel('GHI (W/m²)')
    axes[1].set_ylabel('Module B Output (W)')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nModule A Statistics:")
    print(df_daytime['ModA'].describe())
    print(f"\nModule B Statistics:")
    print(df_daytime['ModB'].describe())
    
    print("✓ Module output analysis complete")
else:
    print("⚠ Module output columns (ModA, ModB) not found in dataset")

## 14. Analysis Summary

In [ ]:
print("\n" + "="*70)
print("ANALYSIS SUMMARY")
print("="*70)

print(f"\n📊 Dataset Overview:")
print(f"  • Total records: {len(df):,}")
print(f"  • Date range: {df.index.min().date()} to {df.index.max().date()}")
print(f"  • Duration: {(df.index.max() - df.index.min()).days} days")
print(f"  • Columns: {len(df.columns)}")

print(f"\n☀️ Solar Irradiance:")
print(f"  • Mean GHI: {df['GHI'].mean():.2f} W/m²")
print(f"  • Max GHI: {df['GHI'].max():.2f} W/m²")
print(f"  • Daytime records (GHI > {DAYTIME_GHI_THRESHOLD}): {len(df_daytime):,} ({len(df_daytime)/len(df)*100:.1f}%)")

print(f"\n🌡️ Temperature:")
print(f"  • Mean ambient temperature: {df['Tamb'].mean():.2f}°C")
print(f"  • Mean module temperature: {df['TModA'].mean():.2f}°C")
print(f"  • Temperature range: {df['Tamb'].min():.2f}°C to {df['Tamb'].max():.2f}°C")

print(f"\n💨 Wind:")
print(f"  • Mean wind speed: {df['WS'].mean():.2f} m/s")
print(f"  • Max wind speed: {df['WS'].max():.2f} m/s")

print(f"\n💧 Humidity:")
print(f"  • Mean relative humidity: {df['RH'].mean():.2f}%")

print(f"\n✓ Analysis complete!")
print("="*70)